# Data Collection:

#### This notebook contains code for data collection. The data will be used to train our JetBots to navigate their environment. We use this code to collect images with the JetBot's camera and place the images in appropriately labeled folders. We have 4 datasets: ``free``, ``blocked``, ``left``, and ``right``.


This code is modified from NVidia's JetBot project here: https://github.com/NVIDIA-AI-IOT/jetbot

George Gorospe of NASA and STARR has modified the data collection notebook here: https://github.com/RFisherIves/STARR

We have also utilized code from DVillevald's JetBot project here: https://github.com/dvillevald/Finding-path-in-maze-of-traffic-cones/ 

#### Step 1: Create a viewer and display the JetBot's camera feed:

In [ ]:
import os
import traitlets
import ipywidgets.widgets as widgets

from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

os.system('systemctl restart nvargus-daemon') # Restart daemon to reset camera

camera = Camera.instance(width=224, height=224) # Set the minimum image size for minimum file size

image = widgets.Image(format='jpeg', width=224, height=224) # Set viewer to the same size as camera image

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg) # update the image displayed from the camera feed

display(image) # show image in viewer

#### Step 2: Create directories to store the images.

In [ ]:
# assign paths for the dataset directories
free_dir = 'dataset_FBLR/free' 
blocked_dir = 'dataset_FBLR/blocked'
left_dir = 'dataset_FBLR/left'
right_dir = 'dataset_FBLR/right'

# call the OS to create our dataset directories if they don't already exist
try:
    os.makedirs(free_dir)
    os.makedirs(blocked_dir)
    os.makedirs(left_dir)
    os.makedirs(right_dir)
    
except FileExistsError:
    print('Directories already exist')

#### Step 3: Create a GUI interface

In [ ]:
# Create a button layout
button_layout_1 = widgets.Layout(width='128px', height='64px')
button_layout_2 = widgets.Layout(width='56px', height='64px')

# Associate buttons with the layout and style them
free_button = widgets.Button(description='Free',   button_style='success', layout=button_layout_1)
left_button = widgets.Button(description='Left',   button_style='info', layout=button_layout_1)
right_button = widgets.Button(description='Right',   button_style='info', layout=button_layout_1)
blocked_button = widgets.Button(description='Blocked',   button_style='danger', layout=button_layout_1)


# Add counters for each dataset
free_count = widgets.IntText(layout=button_layout_2,  value=len(os.listdir(free_dir)))
left_count = widgets.IntText(layout=button_layout_2,  value=len(os.listdir(left_dir)))
right_count = widgets.IntText(layout=button_layout_2,  value=len(os.listdir(right_dir)))
blocked_count = widgets.IntText(layout=button_layout_2,  value=len(os.listdir(blocked_dir)))


# Associate the counters with their button
display(widgets.HBox([free_button, free_count]))
display(widgets.HBox([left_button, left_count]))
display(widgets.HBox([right_button, right_count]))
display(widgets.HBox([blocked_button, blocked_count]))

#### Step 4: Define functions for the buttons

In [ ]:
from uuid import uuid1

# function to save an image as a .jpg
def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

# functions to save an image to our dataset directory
def save_free():
    global free_dir, free_count
    save_snapshot(free_dir) # save an image into the directory
    free_count.value = len(os.listdir(free_dir)) # update the count for that directory
    
def save_blocked():
    global blocked_dir, blocked_count
    save_snapshot(blocked_dir)
    blocked_count.value = len(os.listdir(blocked_dir))
    
def save_left():
    global left_dir, left_count
    save_snapshot(left_dir)
    left_count.value = len(os.listdir(left_dir))
    
def save_right():
    global right_dir, right_count
    save_snapshot(right_dir)
    right_count.value = len(os.listdir(right_dir))

# associate our GUI buttons with our functions
free_button.on_click(lambda x: save_free()) # On mouse click, save an image to the dataset directory
blocked_button.on_click(lambda x: save_blocked())
left_button.on_click(lambda x: save_left())
right_button.on_click(lambda x: save_right())

#### Step 5: Execute interactive code and collect data. Position/maneuver the JetBot to capture images of the appropriate datasets. Place the JetBot in a location where it can't move forward, click ``blocked``. Place the JetBot in a location where it can move forward, click ``free``. Place the JetBot in a location where it needs to turn to avoid becoming blocked, click ``left`` or ``right``. Repeat until each dataset directory contains 100s of corresponding images.

In [ ]:
# display camera feed
display(image)

# arrange buttons
top_box = widgets.HBox([free_count, free_button])
middle_box = widgets.HBox([left_button, left_count, right_button, right_count])
bottom_box = widgets.HBox([blocked_count, blocked_button])

# display buttons
display(top_box)
display(middle_box)
display(bottom_box)

#### Step 6: Move on to the ``train_model_v2.ipynb`` notebook.